In [ ]:
import inference
import time
import os
import inference_e2e_v2 as inference_vocoder
from russian import normalize_russian
from text2phonemesequence import Text2PhonemeSequence
path_to_taco = 'ckpt/2try_200k/ckpt_200000'
path_to_hifigan = 'hifigan/cp_hifigan/GTA/g_00520000'
taco2 = inference.load_model(path_to_taco)
hifigan = inference_vocoder.get_model(path_to_hifigan)
text2phone_model = Text2PhonemeSequence(language='rus', is_cuda=True)

In [ ]:
def get_audio_by_text(text_list, taco2, hifigan, phoneme_model, path_save_mel, path_save_wav):
    for idx, text in enumerate(text_list):
        start = time.time()
        prepared_text = normalize_russian(text)
        print(f"filename: {str(idx) + text[:20]}, normalize_russian time: {time.time() - start}")
        start = time.time()
        prepared_text = phoneme_model.infer_sentence(prepared_text)
        print(f"filename: {str(idx) + text[:20]}, phoneme_model time: {time.time() - start}")
        start = time.time()
        output = inference.infer(prepared_text, taco2)
        print(f"filename: {str(idx) + text[:20]}, taco2 time: {time.time() - start}")
        start = time.time()
        inference.save_mel(output, f'{path_save_mel}{str(idx) + text[:20]}')
        print(f"filename: {str(idx) + text[:20]}, taco2 save_mel time: {time.time() - start}")
        inference_vocoder.inference_by_mel(hifigan, f'{path_save_mel}{str(idx) + text[:20]}.npy', f'{path_save_wav}{str(idx) + text[:20]}.wav')
        print(f"filename: {str(idx) + text[:20]}, full cycle time: {time.time() - start}")

In [ ]:
path_save_mel = 'inference_example/input_mels_dir/'
path_save_wav = 'inference_example/output_wav_dir/'
data = ['Пример 1', 
        'Пример 2']
get_audio_by_text(data, taco2, hifigan, text2phone_model, path_save_mel, path_save_wav)